In [1]:
import pandas as pd
import numpy as np
import random, math

In [2]:
random.seed(786)
np.random.seed(786)

In [3]:
df = pd.read_csv('final_data.csv')
# df1 = df[['compound','neg','neu','pos','Close']]
df1 = df[['compound','neg','neu','pos']]
df2=df[['Close']]
# df = df[['neg','neu','pos','Close']]

In [4]:
from sklearn import preprocessing
min_max_scalar=preprocessing.MinMaxScaler()
data=min_max_scalar.fit_transform(df1)
# X_test=min_max_scalar.fit_transform(X_test)
# Y_train=min_max_scalar.fit_transform(Y_train)
# Y_test=min_max_scalar.fit_transform(Y_test)

In [5]:
df1=pd.DataFrame(data)

In [6]:
df1.head()

,0,1,2,3
0,0.053189,0.494253,0.800429,0.341667
1,0.150989,0.382184,0.864807,0.250000
2,0.027553,0.916667,0.684549,0.183333
3,0.105067,0.514368,0.778970,0.391667
4,0.208367,0.402299,0.821888,0.391667


In [7]:
close_max=df2.max()
df2/=close_max
# df2=np.array(df2)

In [8]:
df2.head()

,Close
0,0.387238
1,0.394951
2,0.404868
3,0.405030
4,0.405062


In [9]:
df1['4']=df2

In [10]:
df1.head()

,0,1,2,3,4
0,0.053189,0.494253,0.800429,0.341667,0.387238
1,0.150989,0.382184,0.864807,0.250000,0.394951
2,0.027553,0.916667,0.684549,0.183333,0.404868
3,0.105067,0.514368,0.778970,0.391667,0.405030
4,0.208367,0.402299,0.821888,0.391667,0.405062


In [11]:
# df['Close']=df['Close']/400

In [12]:
data = df1.values.tolist()

In [13]:
data[:2]

[[0.05318934194590236,
  0.4942528735632184,
  0.8004291845493562,
  0.3416666666666667,
  0.3872379038791846],
 [0.1509890997174001,
  0.38218390804597707,
  0.8648068669527895,
  0.25,
  0.39495090255047477]]

In [14]:
# data = data.tolist()

In [15]:
inner_threshold = 0.005;
rate_var = 0;

final_ansr = []

In [16]:
sigmoid=lambda x:1/(1+math.e**(-x))

In [17]:
def fit_func(rec, i):
    x1, x2, x3, x4, y = i[0], i[1], i[2], i[3], i[4]

    out5 = (x1 * rec['data']['w15']) + (x2 * rec['data']['w25']) + (x3 * rec['data']['w35']) + (x4 * rec['data']['w45']) + rec['data']['o5']
    out5 = sigmoid(out5)

    out6 = (x1 * rec['data']['w16']) + (x2 * rec['data']['w26']) + (x3 * rec['data']['w36']) + (x4 * rec['data']['w46']) + rec['data']['o6']
    out6 = sigmoid(out6)
    
    out7 = (x1 * rec['data']['w17']) + (x2 * rec['data']['w27']) + (x3 * rec['data']['w37']) + (x4 * rec['data']['w47']) + rec['data']['o7']
    out7 = sigmoid(out7)

    out8 = (out5 * rec['data']['w58']) + (out6 * rec['data']['w68']) + (out7 * rec['data']['w78']) + rec['data']['o8']
    out8 = sigmoid(out8)
    
    out9 = (out5 * rec['data']['w59']) + (out6 * rec['data']['w69']) + (out7 * rec['data']['w79']) + rec['data']['o9']
    out9 = sigmoid(out9)

    out10 = (out8 * rec['data']['w810']) + (out9 * rec['data']['w910']) + rec['data']['o10']
    out10 = sigmoid(out10)

    err10 = out10 * (1 - out10) * (y - out10)

    if abs(err10) <= inner_threshold:
        # final_ansr.append(out6 * 20000)
        rec['fitness'] += 1


In [18]:
nList = [];
m = 20;
n = 25;
mutation_prob = 75;
mutation = 0.5;

In [19]:
# ranges for weights & biases
wl = -2;
wr = 2;
bl = -1;
br = 1

In [20]:
total_weights=20
total_biases=6

In [21]:
for i in range(n):
    # floating random weights
    w15, w16, w17, w25, w26,w27, w35, w36,w37, w45, w46,w47, w58,w59,w68,w69,w78,w79, w810, w910 = np.random.uniform(wl, wr, total_weights)
    # floating random biases
    o5, o6, o7,o8, o9, o10 = np.random.uniform(bl, br, total_biases)

    nList.append({'data': {'w15': w15, 'w16': w16, 'w17': w17, 'w25': w25, 'w26': w26, 'w27': w27,
                           'w35': w35,'w36': w36, 'w37': w37, 'w45': w45, 'w46': w46,
                           'w47': w47, 'w58': w58, 'w59': w59, 'w68': w68, 'w69': w69, 'w78': w78,
                           'w79': w79,'w810': w810,'w910': w910, 'o5': o5, 'o6': o6, 'o7': o7,
                           'o8': o8, 'o9': o9, 'o10': o10},
                           'fitness': 0})

In [22]:
# fitness calculation for first time/initial population
for rec in nList:
    for i in data:
        fit_func(rec, i)
        

In [23]:
count = 0;
final_rec = 0
flag = False
stop_at=100000

In [ ]:
while (1):
    count += 1
    mList = []

    print(f"gen: {count}")

    if count > stop_at:
        final_rec = sorted(nList, key=lambda f: f['fitness'], reverse=True)[0]
        break

    rec_num = 0
    for rec in nList:
        # print("rec_index: {}".format(rec_num))
        rec_num += 1
        if rec['fitness'] == len(data):
            print("rec_index: {}".format(rec_num))
            final_rec = rec
            # print('gen: {}'.format(count))
            print('Answer: {}'.format(rec))
            flag = True
            break

    if flag:
        break

    for j in range(int(m / 2)):
        # randomly parent selection(not same)
        # t=temp

        t = random.sample(range(n), 2)

        p1 = nList[t[0]]
        p2 = nList[t[1]]

        # crossover
        crossover_point=int((total_weights+total_biases)/2)
        ch1 = list(p1['data'].values())[:crossover_point] + list(p2['data'].values())[crossover_point:]
        ch2 = list(p2['data'].values())[:crossover_point] + list(p1['data'].values())[crossover_point:]

        # mutation work
        for k in [ch1, ch2]:
            if random.randint(0, 100) <= mutation_prob:
                # get random for mutation in weights
                t = random.sample(range(total_weights), 3)
                # get random for mutation in bias
                t1 = random.randint(total_weights, (total_weights+total_biases)-1)
                # odd,do +ve
                if random.randint(1, 2) == 1:
                    # mutation in weights
                    if (k[t[0]] + mutation) >= wl and (k[t[0]] + mutation) <= wr:
                        k[t[0]] += mutation

                    if (k[t[1]] + mutation) >= wl and (k[t[1]] + mutation) <= wr:
                        k[t[1]] += mutation

                    if (k[t[2]] + mutation) >= wl and (k[t[2]] + mutation) <= wr:
                        k[t[2]] += mutation

                    # mutation in bias
                    if (k[t1] + mutation) >= bl and (k[t1] + mutation) <= br:
                        k[t1] += mutation

                # even,do -ve
                else:
                    # mutation in weights
                    if (k[t[0]] - mutation) >= wl and (k[t[0]] - mutation) <= wr:
                        k[t[0]] -= mutation

                    if (k[t[1]] - mutation) >= wl and (k[t[1]] - mutation) <= wr:
                        k[t[1]] -= mutation

                    if (k[t[2]] - mutation) >= wl and (k[t[2]] - mutation) <= wr:
                        k[t[2]] -= mutation

                    # mutation in bias
                    if (k[t1] - mutation) >= bl and (k[t1] - mutation) <= br:
                        k[t1] -= mutation

                        
        ch1 = {'data': {'w15': ch1[0], 'w16': ch1[1], 'w17': ch1[2], 'w25': ch1[3], 'w26': ch1[4], 'w27': ch1[5],
                        'w35': ch1[6],
                        'w36': ch1[7], 'w37': ch1[8], 'w45': ch1[9], 'w46': ch1[10],
                        'w47': ch1[11], 'w58': ch1[12], 'w59': ch1[13], 'w68': ch1[14], 'w69': ch1[15], 'w78': ch1[16],
                        'w79': ch1[17],
                        'w810': ch1[18], 'w910': ch1[19], 'o5': ch1[20], 'o6': ch1[21],
                        'o7': ch1[22], 'o8': ch1[23], 'o9': ch1[24], 'o10': ch1[25]}, 'fitness': 0}

        ch2 = {'data': {'w15': ch2[0], 'w16': ch2[1], 'w17': ch2[2], 'w25': ch2[3], 'w26': ch2[4], 'w27': ch2[5],
                        'w35': ch2[6],
                        'w36': ch2[7], 'w37': ch2[8], 'w45': ch2[9], 'w46': ch2[10],
                        'w47': ch2[11], 'w58': ch2[12], 'w59': ch2[13], 'w68': ch2[14], 'w69': ch2[15], 'w78': ch2[16],
                        'w79': ch2[17],
                        'w810': ch2[18], 'w910': ch2[19], 'o5': ch2[20], 'o6': ch2[21],
                        'o7': ch2[22], 'o8': ch2[23], 'o9': ch2[24], 'o10': ch2[25]}, 'fitness': 0}


        # calculate fitness of children
        for rec in [ch1, ch2]:
            for i in data:
                fit_func(rec, i)

        mList.append(ch1)
        mList.append(ch2)

    # combine both mList and nList and sort it with respect fit func value,also ovverides the nList,so we used in next iteration

    nList = sorted(nList + mList, key=lambda item: item['fitness'], reverse=True)[:25]


gen: 52064
gen: 52065
gen: 52066
gen: 52067
gen: 52068
gen: 52069
gen: 52070
gen: 52071
gen: 52072
gen: 52073
gen: 52074
gen: 52075
gen: 52076
gen: 52077
gen: 52078
gen: 52079
gen: 52080
gen: 52081
gen: 52082
gen: 52083
gen: 52084
gen: 52085
gen: 52086
gen: 52087
gen: 52088
gen: 52089
gen: 52090
gen: 52091
gen: 52092
gen: 52093
gen: 52094
gen: 52095
gen: 52096
gen: 52097
gen: 52098
gen: 52099
gen: 52100
gen: 52101
gen: 52102
gen: 52103
gen: 52104
gen: 52105
gen: 52106
gen: 52107
gen: 52108
gen: 52109
gen: 52110
gen: 52111
gen: 52112
gen: 52113
gen: 52114
gen: 52115
gen: 52116
gen: 52117
gen: 52118
gen: 52119
gen: 52120
gen: 52121
gen: 52122
gen: 52123
gen: 52124
gen: 52125
gen: 52126
gen: 52127
gen: 52128
gen: 52129
gen: 52130
gen: 52131
gen: 52132
gen: 52133
gen: 52134
gen: 52135
gen: 52136
gen: 52137
gen: 52138
gen: 52139
gen: 52140
gen: 52141
gen: 52142
gen: 52143
gen: 52144
gen: 52145
gen: 52146
gen: 52147
gen: 52148
gen: 52149
gen: 52150
gen: 52151
gen: 52152
gen: 52153
gen: 52154

gen: 52809
gen: 52810
gen: 52811
gen: 52812
gen: 52813
gen: 52814
gen: 52815
gen: 52816
gen: 52817
gen: 52818
gen: 52819
gen: 52820
gen: 52821
gen: 52822
gen: 52823
gen: 52824
gen: 52825
gen: 52826
gen: 52827
gen: 52828
gen: 52829
gen: 52830
gen: 52831
gen: 52832
gen: 52833
gen: 52834
gen: 52835
gen: 52836
gen: 52837
gen: 52838
gen: 52839
gen: 52840
gen: 52841
gen: 52842
gen: 52843
gen: 52844
gen: 52845
gen: 52846
gen: 52847
gen: 52848
gen: 52849
gen: 52850
gen: 52851
gen: 52852
gen: 52853
gen: 52854
gen: 52855
gen: 52856
gen: 52857
gen: 52858
gen: 52859
gen: 52860
gen: 52861
gen: 52862
gen: 52863
gen: 52864
gen: 52865
gen: 52866
gen: 52867
gen: 52868
gen: 52869
gen: 52870
gen: 52871
gen: 52872
gen: 52873
gen: 52874
gen: 52875
gen: 52876
gen: 52877
gen: 52878
gen: 52879
gen: 52880
gen: 52881
gen: 52882
gen: 52883
gen: 52884
gen: 52885
gen: 52886
gen: 52887
gen: 52888
gen: 52889
gen: 52890
gen: 52891
gen: 52892
gen: 52893
gen: 52894
gen: 52895
gen: 52896
gen: 52897
gen: 52898
gen: 52899

gen: 53554
gen: 53555
gen: 53556
gen: 53557
gen: 53558
gen: 53559
gen: 53560
gen: 53561
gen: 53562
gen: 53563
gen: 53564
gen: 53565
gen: 53566
gen: 53567
gen: 53568
gen: 53569
gen: 53570
gen: 53571
gen: 53572
gen: 53573
gen: 53574
gen: 53575
gen: 53576
gen: 53577
gen: 53578
gen: 53579
gen: 53580
gen: 53581
gen: 53582
gen: 53583
gen: 53584
gen: 53585
gen: 53586
gen: 53587
gen: 53588
gen: 53589
gen: 53590
gen: 53591
gen: 53592
gen: 53593
gen: 53594
gen: 53595
gen: 53596
gen: 53597
gen: 53598
gen: 53599
gen: 53600
gen: 53601
gen: 53602
gen: 53603
gen: 53604
gen: 53605
gen: 53606
gen: 53607
gen: 53608
gen: 53609
gen: 53610
gen: 53611
gen: 53612
gen: 53613
gen: 53614
gen: 53615
gen: 53616
gen: 53617
gen: 53618
gen: 53619
gen: 53620
gen: 53621
gen: 53622
gen: 53623
gen: 53624
gen: 53625
gen: 53626
gen: 53627
gen: 53628
gen: 53629
gen: 53630
gen: 53631
gen: 53632
gen: 53633
gen: 53634
gen: 53635
gen: 53636
gen: 53637
gen: 53638
gen: 53639
gen: 53640
gen: 53641
gen: 53642
gen: 53643
gen: 53644

gen: 54299
gen: 54300
gen: 54301
gen: 54302
gen: 54303
gen: 54304
gen: 54305
gen: 54306
gen: 54307
gen: 54308
gen: 54309
gen: 54310
gen: 54311
gen: 54312
gen: 54313
gen: 54314
gen: 54315
gen: 54316
gen: 54317
gen: 54318
gen: 54319
gen: 54320
gen: 54321
gen: 54322
gen: 54323
gen: 54324
gen: 54325
gen: 54326
gen: 54327
gen: 54328
gen: 54329
gen: 54330
gen: 54331
gen: 54332
gen: 54333
gen: 54334
gen: 54335
gen: 54336
gen: 54337
gen: 54338
gen: 54339
gen: 54340
gen: 54341
gen: 54342
gen: 54343
gen: 54344
gen: 54345
gen: 54346
gen: 54347
gen: 54348
gen: 54349
gen: 54350
gen: 54351
gen: 54352
gen: 54353
gen: 54354
gen: 54355
gen: 54356
gen: 54357
gen: 54358
gen: 54359
gen: 54360
gen: 54361
gen: 54362
gen: 54363
gen: 54364
gen: 54365
gen: 54366
gen: 54367
gen: 54368
gen: 54369
gen: 54370
gen: 54371
gen: 54372
gen: 54373
gen: 54374
gen: 54375
gen: 54376
gen: 54377
gen: 54378
gen: 54379
gen: 54380
gen: 54381
gen: 54382
gen: 54383
gen: 54384
gen: 54385
gen: 54386
gen: 54387
gen: 54388
gen: 54389

In [ ]:
for i in data:
    x1, x2, x3, x4, y = i[0], i[1], i[2], i[3], i[4]

    out5 = (x1 * final_rec['data']['w15']) + (x2 * final_rec['data']['w25']) + (x3 * final_rec['data']['w35']) + (x4 * final_rec['data']['w45']) + final_rec['data']['o5']
    out5 = sigmoid(out5)

    out6 = (x1 * final_rec['data']['w16']) + (x2 * final_rec['data']['w26']) + (x3 * final_rec['data']['w36']) + (x4 * final_rec['data']['w46']) + final_rec['data']['o6']
    out6 = sigmoid(out6)
    
    out7 = (x1 * final_rec['data']['w17']) + (x2 * final_rec['data']['w27']) + (x3 * final_rec['data']['w37']) + (x4 * final_rec['data']['w47']) + final_rec['data']['o7']
    out7 = sigmoid(out7)

    out8 = (out5 * final_rec['data']['w58']) + (out6 * final_rec['data']['w68']) + (out7 * final_rec['data']['w78']) + final_rec['data']['o8']
    out8 = sigmoid(out8)
    
    out9 = (out5 * final_rec['data']['w59']) + (out6 * final_rec['data']['w69']) + (out7 * final_rec['data']['w79']) + final_rec['data']['o9']
    out9 = sigmoid(out9)

    out10 = (out8 * final_rec['data']['w810']) + (out9 * final_rec['data']['w910']) + final_rec['data']['o10']
    out10 = sigmoid(out10)

    final_ansr.append(out10*close_max)

In [ ]:
# final_ansr=final_ansr[-len(data):]
df['Y-ENN'] = pd.DataFrame(final_ansr)
df.to_csv('NN-DATA2.csv', index=False)

In [ ]:
final_rec

In [ ]:
from sklearn.metrics import r2_score
r2_score(df[['Close']],df['Y-ENN'])

In [ ]:
df[['Close']].head()

In [ ]:
df['Y-ENN'].head()